In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [74]:
# load dataset. 
# this file contains some works by shakespere as plain text

with open(r'./input.txt') as file:
    text = file.read()

len(text), text[:100]

(1115394,
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

In [75]:
# preparatory work. 
# alphabet could also be called a token dictionary.
# encode and decode are functions to quickly tokenize and detokenize a text.
# lastly, data is split in train and validation set

alphabet = ''.join(sorted(list(set(text))))
alphabet, len(alphabet)

stoi = {s:i for i,s in enumerate(alphabet)}
itos = {i:s for i,s in enumerate(alphabet)}

encode = lambda x: [stoi[i] for i in x]
decode = lambda x: ''.join([itos[i] for i in x])

data = torch.tensor(encode(text), dtype=torch.long)
split_id = int(0.9*len(data))
train_data = data[:split_id]
val_data = data[split_id:]


In [67]:
B,T,C = 4,8,10
nH, sH = 5, 2

tril = torch.tril(torch.ones((T,T)))

key = torch.randn((1,nH,C,sH))
query = torch.randn((1,nH,C,sH))
value = torch.randn((1,nH,C,sH))

x1 = torch.randn((B,T,C))
x = x1.view((B,1,T,C))

k = x @ key
q = x @ query

wei = q @ k.transpose(-1,-2)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = x @ value
out = wei @ v
out = out.permute(0,2,1,3).reshape((B,T,-1))
x1.shape, x.shape, v.shape, wei.shape, out.shape

(torch.Size([4, 8, 10]),
 torch.Size([4, 1, 8, 10]),
 torch.Size([4, 5, 8, 2]),
 torch.Size([4, 5, 8, 8]),
 torch.Size([4, 8, 10]))

In [282]:
# fct that prepares data for training. 
# x is of shape (batchsize, blocksize)
# y is of shape (batchsize, blocksize)
# for every entry in x[i,j], 
# y[i,j] is the prediction given context x[:i,j]
# this structure makes sense given the attention blocks that are 
# introduced later!

blocksize = 8
batchsize = 4
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(train_data)-blocksize, (batchsize,))
    x = torch.stack([train_data[i:i+blocksize] for i in ix])
    y = torch.stack([train_data[i+1:i+blocksize+1] for i in ix])
    return x,y

Xb,Yb = get_batch('train')
print(Xb,Yb,sep='\n')
Xb.shape,Yb.shape

tensor([[46, 39, 58, 46,  1, 61, 53, 59],
        [43,  1, 51, 39, 50, 47, 41, 43],
        [39, 56, 42, 10,  1, 40, 43,  1],
        [53,  1, 51, 43, 56, 47, 58,  1]])
tensor([[39, 58, 46,  1, 61, 53, 59, 52],
        [ 1, 51, 39, 50, 47, 41, 43,  1],
        [56, 42, 10,  1, 40, 43,  1, 45],
        [ 1, 51, 43, 56, 47, 58,  1, 40]])


(torch.Size([4, 8]), torch.Size([4, 8]))

In [157]:
# the most simple prediction model. 
# takes in a single character and predicts the next.
# the embedding provides a prob. distr. for every character in alphabet.
# watch out - torch usually requires (B,C,d1,d2,...), 
# we mostly use (B,d1,d2,...,C), so occasionally some modification are necesarry. 
# forward() specifies how the network works. 
# generate() applies the network to generate text based on its learning.
# notices that in this network the batch AND the time dimension act as
# if they are batch dimension. there is no special meaning to the
# time dimension as of now. 
# that is the reason why we can concatenate these dimensions in forward().


class BigramModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(len(alphabet), len(alphabet))

    def forward(self, x, y=None):
        logits = self.emb(x)

        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)

        return logits, loss
    
    def generate(self, x, max_tokens):
        for _ in range(max_tokens):
            logits, loss = self(x)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            pred = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, pred), dim=1)
        return x


In [249]:
m = BigramModel()
log, los = m(Xb, Yb)

print(decode(m.generate(x=torch.zeros((1,1), dtype=torch.long), max_tokens=50)[0].tolist()))


kx$jaBUnRwWBOe!XYTEDDTfOehYrltVn!LskxGAzi$aYDh$ajQ


In [250]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [269]:
for _ in range(1000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.1561286449432373


In [280]:
print(decode(m.generate(x=torch.zeros((1,1), dtype=torch.long), max_tokens=50)[0].tolist()))


EOxBygS:
In IINCH-to lerd scthar upine ito:
K:

Hd


In [301]:
# version 1
tril = torch.tril(torch.ones((blocksize,blocksize))) 
atmask = tril / tril.sum(axis=1, keepdim=True)


In [304]:
# version 3
tril = torch.tril(torch.ones((blocksize,blocksize)))
wei = torch.zeros((blocksize,blocksize))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
(wei @ xtest).shape

torch.Size([4, 8, 2])

In [320]:
# version 4: self-attention
B,T,C = 4,8,2
head_size = 6
xtest = torch.randn((B,T,C))    # (B,T,C)
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(xtest)      # (B,T,H)
q = query(xtest)    # (B,T,H)
wei = k @ q.transpose(-1,-2) * head_size**-0.5   # (B,T,H) @ (B,H,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(xtest)
out = wei @ v

wei.shape, v.shape, out.shape

(torch.Size([4, 8, 8]), torch.Size([4, 8, 6]), torch.Size([4, 8, 6]))